In [ ]:
import os
import subprocess
from threading import Timer
from queue import Queue

ROOT_DIR = "/content"
REPO_URL = "https://huggingface.co/spaces/Linaqruf/animagine-xl"
REPO_DIR = os.path.join(ROOT_DIR, "cagliostro-webui")
NGROK_TOKEN = ""

os.environ["HF_TOKEN"] = ""
os.environ["IS_COLAB"] = "1"
os.environ["MODEL"] = "https://huggingface.co/cagliostrolab/animagine-xl-3.0/blob/main/animagine-xl-3.0.safetensors"
os.environ["CACHE_EXAMPLES"] = "1"

def clone(url, dir, branch=None):
    subprocess.run(["git", "clone", url, dir], check=True)
    if branch:
        subprocess.run(["git", "checkout", branch], cwd=dir, check=True)

def install_deps(dir):
    subprocess.run(["pip", "install", "-r", "requirements.txt"], cwd=dir, check=True)

def ngrok_tunnel(port,queue,auth_token):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port)
    queue.put(url)

def main():
    if not os.path.exists(REPO_DIR):
        print(f"Cloning Repository to {REPO_DIR}")
        clone(REPO_URL, REPO_DIR)
    print(f"Installing required python libraries")
    install_deps(REPO_DIR)
    print("Done!")

    os.chdir(REPO_DIR)
    
    if NGROK_TOKEN:
        try:
            from pyngrok import conf,ngrok
        except:
            !pip install -qqqq --upgrade setuptools
            !pip install -qqqq -U pyngrok
            from pyngrok import conf,ngrok
                                
        ngrok_output_queue = Queue()
        ngrok_thread = Timer(2, ngrok_tunnel, args=(7860, ngrok_output_queue, NGROK_TOKEN))
        ngrok_thread.start()
        ngrok_thread.join()
        
        print(ngrok_output_queue.get())  
    
    !python app.py

if __name__ == "__main__":
    main()